# Reinforcement Learning, 2024-01
## Tarea 4 - Algoritmos de aprendizaje por refuerzo con aproximación de funciones

> Daniel Villar González, 201923374.  
> Daniel Alvarez, 201911320.

## **Descripción de tarea**

Considere el problema Cart Pole implementado en el entorno de Gymnasium descrito. El objetivo de este taller es comparar algoritmos de RL tabulares con sus contrapartes que utilizan aproximación de funciones.

### ***Requerimientos***
1. La recompensa vista en un estado terminal es cero.
2. El siguiente estado visto por un agente en un estado terminal es igual al mismo, cumpliendo con las dinámicas del MDP.
3. Para este caso, como existe aprendizaje, no se conocen de primera mano las probabilidades de transición, por lo que el agente debe aprender.
4. Existen varios episodios donde se actualiza la matriz Q, con el fin de conocer los mayores valores del par estado-acción.

## **Librerias**

In [3]:
import numpy as np
import random
import gym

In [4]:
env = gym.make("CartPole-v1")

def Random_games():
    # Each of this episode is its own game.
    for episode in range(10):
        env.reset()
        # this is each frame, up to 500...but we wont make it that far with random.
        for t in range(500):
            # This will display the environment
            # Only display if you really want to see it.
            # Takes much longer to display it.
            env.render()
            
            # This will just create a sample action in any environment.
            # In this environment, the action can be 0 or 1, which is left or right
            action = env.action_space.sample()

            # this executes the environment with an action, 
            # and returns the observation of the environment, 
            # the reward, if the env is over, and other info.
            next_state, reward, done, info = env.step(action)
            
            # lets print everything in one line:
            print(t, next_state, reward, done, info, action)
            if done:
                break
                
Random_games()

c:\Users\danie\AppData\Local\Programs\Python\Python38\lib\site-packages\gym\envs\classic_control\cartpole.py:211: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(
c:\Users\danie\AppData\Local\Programs\Python\Python38\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


ValueError: too many values to unpack (expected 4)